下面是在http://58.241.217.181:15111/notebooks/work/ml/gnt-a/notebooks/code/customized-model-A.ipynb#
的基础上，参照：
https://www.tensorflow.org/versions/master/tutorials/layers
来制作CNN，用于识别GNT.



In [ ]:
import tensorflow as tf
import numpy as np
import os
import random
from numpy import array

from skimage.io import imread, imsave
from skimage.color import rgb2gray
from skimage.transform import resize
from skimage.exposure import adjust_gamma

IMAGE_SIZE = 64
# CHARSET_SIZE = 3755
CHARSET_SIZE = 37

def getRndGamma():
    rnd1 = np.random.random_sample()
    rnd2 = np.random.random_sample()+0.001
    rnd3 = np.random.random_sample()*100

    if (rnd1 < 1/3.0):
        return rnd3
    elif (rnd1 < 2/3.0):
        return 1
    else:
        return rnd2


def input(dataset):
    return dataset.images, dataset.labels

def cnn_model_fn(features, labels, mode):
  """Model function for CNN."""
  # Input Layer
  input_layer = tf.reshape(features["x"], [-1, 64, 64, 1])
  print ('shape of input_layer: ', input_layer)
  # with batch_size =100, shape should be: [100, 28, 28, 1]
  # shape of input_layer:  Tensor("Reshape:0", shape=(100, 64, 64, 1), dtype=float32)

  # Convolutional Layer #1
  conv1 = tf.layers.conv2d(
      inputs=input_layer,
      filters=32,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)
  print ('shape of conv1: ', conv1)
  # shape of conv1:  Tensor("conv2d/Relu:0", shape=(100, 64, 64, 32), dtype=float32)
    
  # Pooling Layer #1
  pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)

  # Convolutional Layer #2 and Pooling Layer #2
  conv2 = tf.layers.conv2d(
      inputs=pool1,
      filters=64,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)
  pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)
  print ('shape of conv2: ', conv2, '; and shape of pool2 is: ', pool2)

  # Dense Layer
  pool2_flat = tf.reshape(pool2, [-1, 16 * 16 * 64])
  dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)
  dropout = tf.layers.dropout(
      inputs=dense, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)

  # Logits Layer
  logits = tf.layers.dense(inputs=dropout, units=CHARSET_SIZE)

  predictions = {
      # Generate predictions (for PREDICT and EVAL mode)
      "classes": tf.argmax(input=logits, axis=1),
      # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
      # `logging_hook`.
      "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
  }

  if mode == tf.estimator.ModeKeys.PREDICT:
    return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

  # Calculate Loss (for both TRAIN and EVAL modes)
  loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

  # Configure the Training Op (for TRAIN mode)
  if mode == tf.estimator.ModeKeys.TRAIN:
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
    train_op = optimizer.minimize(
        loss=loss,
        global_step=tf.train.get_global_step())
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

  # Add evaluation metrics (for EVAL mode)
  eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(
          labels=labels, predictions=predictions["classes"])}
  return tf.estimator.EstimatorSpec(
      mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)


class DataSetLoader:
    def __init__(self, data_dir):
        # Set FLAGS.charset_size to a small value if available computation power is limited.
        truncate_path = data_dir + ('%05d' % CHARSET_SIZE)
        print('Now processing path: ', truncate_path)
        image_names = []
        for root, sub_folder, file_list in os.walk(data_dir):
            if root < truncate_path:
                image_names += [os.path.join(root, file_path) for file_path in file_list]
        random.shuffle(image_names)
        self.labels = [int(file_name[len(data_dir):].split(os.sep)[0]) for file_name in image_names]
        images_rgb = [imread(file_name) for file_name in image_names]
        image_resized = [resize(image, (IMAGE_SIZE, IMAGE_SIZE)) for image in images_rgb]
        image_gamma_adjust = [adjust_gamma(image, 1) for image in image_resized]

        self.images = [rgb2gray(item) for item in image_gamma_adjust]
        print ('self.images: ', self.images[0].dtype)
        self.images = np.float32(self.images)
        print ('self.images: ', self.images[0].dtype)

        # convert list to numpy array
        self.images = array(self.images)
        self.labels = array(self.labels)

    
train_data = DataSetLoader(data_dir='../data/train_/')
print ('Train data loaded ...')


# Specify feature
feature_columns = [tf.feature_column.numeric_column("x", shape=[IMAGE_SIZE, IMAGE_SIZE])]



# Build CNN with customized function ...
classifier = tf.estimator.Estimator(
    model_fn=cnn_model_fn,
   model_dir='../dfs/checkpoint/customized_model-c')

    

# Define the training inputs
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": input(train_data)[0]},
    y=input(train_data)[1],
    num_epochs=None,
    batch_size=100,
    shuffle=True
)
print ('Begin to train ...')

classifier.train(input_fn=train_input_fn, steps=40000)
print ('Train done ...')

test_data = DataSetLoader(data_dir='../data/test_/')
print ('Test data loaded ...')

# Define the test inputs
test_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": input(test_data)[0]},
    y=input(test_data)[1],
    num_epochs=1,
    shuffle=False
)

# Evaluate accuracy
accuracy_score = classifier.evaluate(input_fn=test_input_fn)["accuracy"]
print("\nTest Accuracy: {0:f}%\n".format(accuracy_score*100))

Now processing path:  ../data/train_/00037
self.images:  float64
self.images:  float32
Train data loaded ...
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '../dfs/checkpoint/customized_model-c', '_tf_random_seed': 1, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_save_checkpoints_steps': None, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100}
Begin to train ...
shape of input_layer:  Tensor("Reshape:0", shape=(100, 64, 64, 1), dtype=float32)
shape of conv1:  Tensor("conv2d/Relu:0", shape=(100, 64, 64, 32), dtype=float32)
shape of conv2:  Tensor("conv2d_2/Relu:0", shape=(100, 32, 32, 64), dtype=float32) ; and shape of pool2 is:  Tensor("max_pooling2d_2/MaxPool:0", shape=(100, 16, 16, 64), dtype=float32)
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from ../dfs/checkpoint/customized_model-c/model.ckpt-255101
INFO:tensorflow:Saving 

INFO:tensorflow:global_step/sec: 1.13281
INFO:tensorflow:loss = 0.000396507, step = 261402 (88.277 sec)
INFO:tensorflow:global_step/sec: 1.1354
INFO:tensorflow:loss = 0.000993133, step = 261502 (88.074 sec)
INFO:tensorflow:global_step/sec: 1.12678
INFO:tensorflow:loss = 0.000898494, step = 261602 (88.748 sec)
INFO:tensorflow:global_step/sec: 1.135
INFO:tensorflow:loss = 0.00031136, step = 261702 (88.107 sec)
INFO:tensorflow:global_step/sec: 1.13003
INFO:tensorflow:loss = 0.00156596, step = 261802 (88.492 sec)
INFO:tensorflow:global_step/sec: 1.13006
INFO:tensorflow:loss = 0.000273461, step = 261902 (88.492 sec)
INFO:tensorflow:Saving checkpoints for 261967 into ../dfs/checkpoint/customized_model-c/model.ckpt.
INFO:tensorflow:global_step/sec: 1.11922
INFO:tensorflow:loss = 0.00345932, step = 262002 (89.348 sec)
INFO:tensorflow:global_step/sec: 1.12938
INFO:tensorflow:loss = 0.000644465, step = 262102 (88.544 sec)
INFO:tensorflow:global_step/sec: 1.1339
INFO:tensorflow:loss = 0.000722043

-> 关于kernel size:
TIP: If filter height and width have the same value, you can instead specify a single integer for kernel_size—e.g., kernel_size=5.

tensorflow 官网上MNIST示例，是5*5： https://www.tensorflow.org/versions/master/tutorials/layers

Very small filter sizes will capture very fine details of the image. On the other hand having a bigger filter size will leave out minute details in the image.
如果图片细节重要，则需要kernel size小
https://www.quora.com/How-can-I-decide-the-kernel-size-output-maps-and-layers-of-CNN

这个好像也没有固定答案。一般建议 3*3 , 5*5 或7*7。 一般而言，3*3可能会好些。
https://stats.stackexchange.com/questions/296679/what-does-kernel-size-mean



-> 什么叫 activation function

https://towardsdatascience.com/activation-functions-and-its-types-which-is-better-a9a5310cc8f

http://www.holehouse.org/mlclass/06_Logistic_Regression_files/Image%20[1].png



http://www.holehouse.org/mlclass/
By activation, we mean the value which is computed and output by that node


    Neural networks were developed as a way to simulate networks of neurones
    
    This is an artificial neurone with a sigmoid (logistic) activation function
    关于激活函数(activation function)，其实说来话长。首先，需要知道神经网络的由来
    
    
神经网络，就是对人脑神经元工作的模拟:

![神经元](http://www.holehouse.org/mlclass/08_Neural_Networks_Representation_files/Image%20[2].png)
![神经网络](http://www.holehouse.org/mlclass/08_Neural_Networks_Representation_files/Image%20[4].png)

可以把上面第2个图的圈圈，想像成一个个神经元。

当然，圈圈都是一个个数；那，这些数怎么算呢？第一层，叫输入层；当然，不论哪层的哪个神经元，都应该有用。那怎么叫有用呢？就是激活它，就是通过它，得到下一层神经元（圈圈）的值；如何激活它？就是使用一个函数，把这个被激活的神经元，或圈圈的数，代到这个函数中；当然，这个函数，就被称为激活函数 （activation function）了。

常见的激活函数，就是大名鼎鼎的sigmoid函数
![sigmoid函数](http://www.holehouse.org/mlclass/06_Logistic_Regression_files/Image%20[1].png)


sigmoid 出场
![sigmoid函数图像](http://www.holehouse.org/mlclass/06_Logistic_Regression_files/Image%20[2].png)
这个函数的目的，就是将无论什么输入，都弄到0和1之间。即这个函数的y值，就是在0，1之间。





自定义CNN雏形完成：
INFO:tensorflow:Starting evaluation at 2018-04-26-06:35:15
INFO:tensorflow:Restoring parameters from ../dfs/checkpoint/customized_model-c/model.ckpt-1000
INFO:tensorflow:Finished evaluation at 2018-04-26-06:35:21
INFO:tensorflow:Saving dict for global step 1000: accuracy = 0.106335, global_step = 1000, loss = 3.58817

Test Accuracy: 10.633484%

INFO:tensorflow:Starting evaluation at 2018-04-26-11:38:19
INFO:tensorflow:Restoring parameters from ../dfs/checkpoint/customized_model-c/model.ckpt-11101
INFO:tensorflow:Finished evaluation at 2018-04-26-11:38:24
INFO:tensorflow:Saving dict for global step 11101: accuracy = 0.78552, global_step = 11101, loss = 0.791613

Test Accuracy: 78.552037%

10000步时，精度不错。

13000时：
INFO:tensorflow:Starting evaluation at 2018-04-26-12:14:59
INFO:tensorflow:Restoring parameters from ../dfs/checkpoint/customized_model-c/model.ckpt-13101
INFO:tensorflow:Finished evaluation at 2018-04-26-12:15:04
INFO:tensorflow:Saving dict for global step 13101: accuracy = 0.80543, global_step = 13101, loss = 0.680529

Test Accuracy: 80.542988%

INFO:tensorflow:Starting evaluation at 2018-04-26-13:40:56
INFO:tensorflow:Restoring parameters from ../dfs/checkpoint/customized_model-c/model.ckpt-15101
INFO:tensorflow:Finished evaluation at 2018-04-26-13:41:02
INFO:tensorflow:Saving dict for global step 15101: accuracy = 0.826697, global_step = 15101, loss = 0.62195

Test Accuracy: 82.669681%
 
 INFO:tensorflow:Starting evaluation at 2018-04-27-10:11:28
INFO:tensorflow:Restoring parameters from ../dfs/checkpoint/customized_model-c/model.ckpt-95101
INFO:tensorflow:Finished evaluation at 2018-04-27-10:11:35
INFO:tensorflow:Saving dict for global step 95101: accuracy = 0.893665, global_step = 95101, loss = 0.469633

Test Accuracy: 89.366513%

INFO:tensorflow:Starting evaluation at 2018-04-29-02:00:37
INFO:tensorflow:Restoring parameters from ../dfs/checkpoint/customized_model-c/model.ckpt-115101
INFO:tensorflow:Finished evaluation at 2018-04-29-02:00:43
INFO:tensorflow:Saving dict for global step 115101: accuracy = 0.893665, global_step = 115101, loss = 0.471583

Test Accuracy: 89.366513%

下面仅做测试：

INFO:tensorflow:Starting evaluation at 2018-04-29-04:28:24
INFO:tensorflow:Restoring parameters from ../dfs/checkpoint/customized_model-c/model.ckpt-125101
INFO:tensorflow:Finished evaluation at 2018-04-29-04:28:30
INFO:tensorflow:Saving dict for global step 125101: accuracy = 0.895475, global_step = 125101, loss = 0.492467

Test Accuracy: 89.547509%

INFO:tensorflow:Starting evaluation at 2018-04-29-13:00:24
INFO:tensorflow:Restoring parameters from ../dfs/checkpoint/customized_model-c/model.ckpt-145101
INFO:tensorflow:Finished evaluation at 2018-04-29-13:00:30
INFO:tensorflow:Saving dict for global step 145101: accuracy = 0.893665, global_step = 145101, loss = 0.504544

Test Accuracy: 89.366513%

INFO:tensorflow:Starting evaluation at 2018-05-01-01:01:11
INFO:tensorflow:Restoring parameters from ../dfs/checkpoint/customized_model-c/model.ckpt-245101
INFO:tensorflow:Finished evaluation at 2018-05-01-01:01:17
INFO:tensorflow:Saving dict for global step 245101: accuracy = 0.897738, global_step = 245101, loss = 0.524077

Test Accuracy: 89.773756%

这个用了10万步，只前进了0.4%，看来，应该是难有进步了。想达到97%，是不大可能了。
解决的方法，可能是应用gamma随机校正，来增加训练数据的复杂性。可能会提高准确度。

加上了gamma 校正，再训练1万步，看看效果：


INFO:tensorflow:Starting evaluation at 2018-05-01-08:30:57
INFO:tensorflow:Restoring parameters from ../dfs/checkpoint/customized_model-c/model.ckpt-255101
INFO:tensorflow:Finished evaluation at 2018-05-01-08:31:03
INFO:tensorflow:Saving dict for global step 255101: accuracy = 0.894118, global_step = 255101, loss = 0.540363

Test Accuracy: 89.411765%
还是不行啊。

再来4万步看看。


In [2]:
# from: https://www.kaggle.com/jeffcarp/example-save-and-load-a-tensorflow-model

import tensorflow as tf
import numpy as np
import os
import random
from numpy import array

from skimage.io import imread, imsave
from skimage.color import rgb2gray
from skimage.transform import resize

from tensorflow.contrib.learn.python.learn.estimators import run_config
from tensorflow.contrib.training.python.training import hparam

IMAGE_SIZE = 64
# CHARSET_SIZE = 3755
CHARSET_SIZE = 37

def input(dataset):
    return dataset.images, dataset.labels

def cnn_model_fn(features, labels, mode):
  """Model function for CNN."""
  # Input Layer
  input_layer = tf.reshape(features["x"], [-1, 64, 64, 1])
  print ('shape of input_layer: ', input_layer)
  # with batch_size =100, shape should be: [100, 28, 28, 1]
  # shape of input_layer:  Tensor("Reshape:0", shape=(100, 64, 64, 1), dtype=float32)

  # Convolutional Layer #1
  conv1 = tf.layers.conv2d(
      inputs=input_layer,
      filters=32,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)
  print ('shape of conv1: ', conv1)
  # shape of conv1:  Tensor("conv2d/Relu:0", shape=(100, 64, 64, 32), dtype=float32)
    
  # Pooling Layer #1
  pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)

  # Convolutional Layer #2 and Pooling Layer #2
  conv2 = tf.layers.conv2d(
      inputs=pool1,
      filters=64,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)
  pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)
  print ('shape of conv2: ', conv2, '; and shape of pool2 is: ', pool2)

  # Dense Layer
  pool2_flat = tf.reshape(pool2, [-1, 16 * 16 * 64])
  dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)
  dropout = tf.layers.dropout(
      inputs=dense, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)

  # Logits Layer
  logits = tf.layers.dense(inputs=dropout, units=CHARSET_SIZE)

  predictions = {
      # Generate predictions (for PREDICT and EVAL mode)
      "classes": tf.argmax(input=logits, axis=1),
      # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
      # `logging_hook`.
      "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
  }

  if mode == tf.estimator.ModeKeys.PREDICT:
    return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

  # Calculate Loss (for both TRAIN and EVAL modes)
  loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

  # Configure the Training Op (for TRAIN mode)
  if mode == tf.estimator.ModeKeys.TRAIN:
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
    train_op = optimizer.minimize(
        loss=loss,
        global_step=tf.train.get_global_step())
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

  # Add evaluation metrics (for EVAL mode)
  eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(
          labels=labels, predictions=predictions["classes"])}
  return tf.estimator.EstimatorSpec(
      mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

class DataSetLoader:
    def __init__(self, data_dir):
        # Set CHARSET_SIZE to a small value if available computation power is limited.
        truncate_path = data_dir + ('%05d' % CHARSET_SIZE)
        print('Now processing path: ', truncate_path)
        image_names = []
        for root, sub_folder, file_list in os.walk(data_dir):
            if root < truncate_path:
                image_names += [os.path.join(root, file_path) for file_path in file_list]
        random.shuffle(image_names)
        self.labels = [int(file_name[len(data_dir):].split(os.sep)[0]) for file_name in image_names]
        images_rgb = [imread(file_name) for file_name in image_names]
        image_resized = [resize(image, (IMAGE_SIZE, IMAGE_SIZE)) for image in images_rgb]
        self.images = [rgb2gray(item) for item in image_resized]
        self.images = np.float32(self.images)
        # or else: TypeError: Value passed to parameter 'input' has DataType float64 not in list of allowed values: float16, float32

        # convert list to numpy array
        self.images = array(self.images)
        self.labels = array(self.labels)

# Specify feature
feature_columns = [tf.feature_column.numeric_column("x", shape=[IMAGE_SIZE, IMAGE_SIZE])]

# Build CNN with customized function ...
classifier = tf.estimator.Estimator(
    model_fn=cnn_model_fn,
   model_dir='../dfs/checkpoint/customized_model-c')



# MODEL_DIR = "../dfs/checkpoint/customized_model-c"
# model_from_checkpoint = make_estimator(MODEL_DIR)

test_data = DataSetLoader(data_dir='../data/test_/')
# Define the test inputs
test_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": input(test_data)[0]},
    y=input(test_data)[1],
    num_epochs=1,
    shuffle=False
)

predict_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": input(test_data)[0]},
    num_epochs=1,
    shuffle=False)

# Evaluate accuracy
accuracy_score = classifier.evaluate(input_fn=test_input_fn)["accuracy"]
print("\nTest Accuracy: {0:f}%\n".format(accuracy_score*100))

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '../dfs/checkpoint/customized_model-c', '_tf_random_seed': 1, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_save_checkpoints_steps': None, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100}
Now processing path:  ../data/test_/00037
shape of input_layer:  Tensor("Reshape:0", shape=(?, 64, 64, 1), dtype=float32)
shape of conv1:  Tensor("conv2d/Relu:0", shape=(?, 64, 64, 32), dtype=float32)
shape of conv2:  Tensor("conv2d_2/Relu:0", shape=(?, 32, 32, 64), dtype=float32) ; and shape of pool2 is:  Tensor("max_pooling2d_2/MaxPool:0", shape=(?, 16, 16, 64), dtype=float32)
INFO:tensorflow:Starting evaluation at 2018-05-01-01:01:11
INFO:tensorflow:Restoring parameters from ../dfs/checkpoint/customized_model-c/model.ckpt-245101
INFO:tensorflow:Finished evaluation at 2018-05-01-01:01:17
INFO:tensorflow:Saving dict for global step